In [ ]:
## set covid19_datasciente as path for python find bulletin package
import sys
from pathlib import Path
from os import getcwd
sys.path.append(str(Path(getcwd()).parent.parent)) ## ../../covid19datascience* <- set parent level here

In [ ]:
import pandas as pd
from epiweeks import Week
from datetime import timedelta, date
from os.path import join

from bulletin import default_input, default_output, root, hoje, data_comeco_pandemia
from bulletin.services.metabase import Metabase
from bulletin.systems.notifica import Notifica
from bulletin.utils.normalize import trim_overspace, normalize_text

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 300)

In [ ]:
nt = Notifica()

In [ ]:
nt.load('notifica', compress=False)

In [ ]:
exames = nt.tables['exame'].sort_values('id')
metodos = nt.tables['metodo']

In [ ]:
nt.df = nt.df.loc[nt.df['data_notificacao'].notna()]

In [ ]:
nt.df['SE'] = nt.df['data_notificacao'].apply(lambda x: Week.fromdate(x).week)
nt.df['ano'] = nt.df['data_notificacao'].apply(lambda x: Week.fromdate(x).year)
nt.df['mes'] = nt.df['data_notificacao'].dt.month
nt.df['dia'] = nt.df['data_notificacao'].dt.day

In [ ]:
nt.tables['metodo']

In [ ]:
rt_pcr = nt.df.loc[nt.df['metodo']==1]

In [ ]:
tabela_rt_pcr = rt_pcr.pivot_table(index=['ano','SE'], columns=['classificacao_final'], values='id', aggfunc='count', fill_value=0)
tabela_rt_pcr = tabela_rt_pcr.rename(columns=nt.tables['classificacao_final'].set_index('id')['classificacao_final'],level='classificacao_final')
tabela_rt_pcr['Total'] = tabela_rt_pcr.sum(axis=1)
tabela_rt_pcr['Positividade'] = tabela_rt_pcr['Caso confirmado'] / tabela_rt_pcr['Total']
ax = tabela_rt_pcr[['Caso confirmado','Caso descartado', 'Total']].rolling(1).mean().plot()
tabela_rt_pcr[['Positividade']].rolling(7).mean().plot(secondary_y=True, ax=ax, ls=':', figsize=(20,12))

In [ ]:
tabela_rt_pcr = rt_pcr.pivot_table(index=['ano','SE'], columns=['classificacao_final'], values='id', aggfunc='count', fill_value=0)
tabela_rt_pcr = tabela_rt_pcr.rename(columns=nt.tables['classificacao_final'].set_index('id')['classificacao_final'],level='classificacao_final')
tabela_rt_pcr['Total'] = tabela_rt_pcr.sum(axis=1)
tabela_rt_pcr['Positividade'] = tabela_rt_pcr['Caso confirmado'] / tabela_rt_pcr['Total']
ax = tabela_rt_pcr[['Caso confirmado','Caso descartado', 'Total']].rolling(1).mean().plot()
tabela_rt_pcr[['Positividade']].rolling(7).mean().plot(secondary_y=True, ax=ax, ls=':', figsize=(20,12))

In [ ]:
outros_testes = nt.df.loc[nt.df['metodo']!=1]

In [ ]:
tabela_outros_testes = outros_testes.pivot_table(index=['ano','SE'], columns=['classificacao_final'], values='id', aggfunc='count', fill_value=0)
tabela_outros_testes = tabela_outros_testes.rename(columns=nt.tables['classificacao_final'].set_index('id')['classificacao_final'],level='classificacao_final')
tabela_outros_testes['Total'] = tabela_outros_testes.sum(axis=1)
tabela_outros_testes['Positividade'] = tabela_outros_testes['Caso confirmado'] / tabela_outros_testes['Total']
ax = tabela_outros_testes[['Caso confirmado','Caso descartado', 'Total']].rolling(1).mean().plot()
tabela_outros_testes[['Positividade']].rolling(7).mean().plot(secondary_y=True, ax=ax, ls=':', figsize=(20,12))

In [ ]:
tabela_rt_pcr

In [ ]:
tabela_outros_testes